Fonction pour nettoyer la base des licenciés par commune :


In [ ]:
#Importation des modules
import numpy as np
import pandas as pd
import seaborn as sns
import pyproj
import fiona
import geopandas

In [ ]:
### fonction pour nettoyer la base lic_comm :

def nettoyer_lic_comm(chemin_acces_lic_comm, chemin_acces_pop_dep) :
    #Importation de la base
    lic_comm = pd.read_csv(chemin_acces_lic_comm, encoding="utf-8", sep=";")

    #Conversion des variables de type object en character

    lic_comm['Département'] = lic_comm['Département'].astype('string')
    lic_comm['Code Commune'] = lic_comm['Code Commune'].astype('string')
    lic_comm['Commune'] = lic_comm['Commune'].astype('string')
    lic_comm['Code QPV'] = lic_comm['Code QPV'].astype('string')
    lic_comm['Nom QPv'] = lic_comm['Nom QPV'].astype('string')
    lic_comm['Région'] = lic_comm['Région'].astype('string')
    lic_comm['Code'] = lic_comm['Code'].astype('string')
    lic_comm['Fédération'] = lic_comm['Fédération'].astype('string')
    lic_comm['Statut géo'] = lic_comm['Statut géo'].astype('string')

    #Enlever les données hors champ de la base lic_comm : variables qui contiennent un 2 ou 3 dans Statut géo

    lic_comm[lic_comm['Statut géo'].str.contains('2')]

    lic_comm = lic_comm[~lic_comm['Statut géo'].str.contains('2')]
    lic_comm = lic_comm[~lic_comm['Statut géo'].str.contains('3')]

    # Modification de la variable 'Département' : nb qui commencent pas par 0 et inf à 10=> leur rejouter un 0
    for i in range(1,10) :
        i = str(i)
        lic_comm['Département'] = np.where(lic_comm['Département'] == i, '0' + i, lic_comm['Département'])


    lic_comm['Département'] = lic_comm['Département'].astype('string') #Variable Département reconvertie en character


    #Enlever les lignes où apprait la mention 'NR' dans la variable commune :
    lic_comm = lic_comm.dropna(subset=['Commune'])

    ### Ajouter les variables population du dépratement à partir du fichier pop_dep_2019
    pop_dep = pd.read_excel(chemin_acces_pop_dep, sheet_name=1, skiprows=7) 

    #Conversion des variables 'object' en string
    pop_dep['Code département'] = pop_dep['Code département'].astype('string')
    pop_dep['Nom du département'] = pop_dep['Nom du département'].astype('string')

    #selectiondes variables d'intérêt
    pop_dep = pop_dep[['Nom de la région', 'Code département', 'Nom du département', 'Population municipale', 'Population totale']] #Faudra décider si on garde pop totale ou minicipale
    
    # On utilise un melt pour réduire la largeur du df comme trop de colonnes : à la place, on rajoute les variables 'sexe' et 'tranche_age', qui permettront de 
# faire des groupby et faciliter la manipulation de la base

    #On groupe la base par département, qui est notre échelle d'analyse
    lic_comm = lic_comm[['Département', 'Code', 'Fédération', 'Région', 'F - 1 à 4 ans',
       'F - 5 à 9 ans', 'F - 10 à 14 ans', 'F - 15 à 19 ans',
       'F - 20 à 24 ans', 'F - 25 à 29 ans', 'F - 30 à 34 ans',
       'F - 35 à 39 ans', 'F - 40 à 44 ans', 'F - 45 à 49 ans',
       'F - 50 à 54 ans', 'F - 55 à 59 ans', 'F - 60 à 64 ans',
       'F - 65 à 69 ans', 'F - 70 à 74 ans', 'F - 75 à 79 ans',
       'F - 80 à 99 ans', 'F - NR', 'H - 1 à 4 ans', 'H - 5 à 9 ans',
       'H - 10 à 14 ans', 'H - 15 à 19 ans', 'H - 20 à 24 ans',
       'H - 25 à 29 ans', 'H - 30 à 34 ans', 'H - 35 à 39 ans',
       'H - 40 à 44 ans', 'H - 45 à 49 ans', 'H - 50 à 54 ans',
       'H - 55 à 59 ans', 'H - 60 à 64 ans', 'H - 65 à 69 ans',
       'H - 70 à 74 ans', 'H - 75 à 79 ans', 'H - 80 à 99 ans'
       ]].groupby([ 'Département', 'Code', 'Fédération', 'Région']).sum().reset_index()
    

    lic_comm = lic_comm.melt(id_vars=['Département', 'Code', 'Fédération', 'Région'],
                         value_vars= ['F - 1 à 4 ans',
       'F - 5 à 9 ans', 'F - 10 à 14 ans', 'F - 15 à 19 ans',
       'F - 20 à 24 ans', 'F - 25 à 29 ans', 'F - 30 à 34 ans',
       'F - 35 à 39 ans', 'F - 40 à 44 ans', 'F - 45 à 49 ans',
       'F - 50 à 54 ans', 'F - 55 à 59 ans', 'F - 60 à 64 ans',
       'F - 65 à 69 ans', 'F - 70 à 74 ans', 'F - 75 à 79 ans',
       'F - 80 à 99 ans', 'H - 1 à 4 ans', 'H - 5 à 9 ans',
       'H - 10 à 14 ans', 'H - 15 à 19 ans', 'H - 20 à 24 ans',
       'H - 25 à 29 ans', 'H - 30 à 34 ans', 'H - 35 à 39 ans',
       'H - 40 à 44 ans', 'H - 45 à 49 ans', 'H - 50 à 54 ans',
       'H - 55 à 59 ans', 'H - 60 à 64 ans', 'H - 65 à 69 ans',
       'H - 70 à 74 ans', 'H - 75 à 79 ans', 'H - 80 à 99 ans'], var_name = 'categorie',  value_name='nb_licencies')
    

    lic_comm['tranche_age'] = 'Ensemble' #Valeur par defaut, pour qu'elle soit bien condiérée comme une variale de type caractère
    lic_comm['sexe'] = "Ensemble" #Pareil
#lic_comm['qpv'] = 'False' #Qpv : nb de licencies vivant dans un qartier prioritaire des politiques de la ville


    #Ajout variable 'tranche_age' : au total 9 catégories d'âge : ici, aggrégation de certaines tranches d'âge (les pus agés) 
    lic_comm['tranche_age'] = np.where(lic_comm['categorie'].str.contains("1 à 4"), "1-4", lic_comm['tranche_age'])
    lic_comm['tranche_age'] = np.where(lic_comm['categorie'].str.contains("5 à 9"), "5-9", lic_comm['tranche_age'])
    lic_comm['tranche_age'] = np.where(lic_comm['categorie'].str.contains("10 à 14"), "10-14", lic_comm['tranche_age'])
    lic_comm['tranche_age'] = np.where(lic_comm['categorie'].str.contains("15 à 19"), "15-19", lic_comm['tranche_age'])
    lic_comm['tranche_age'] = np.where(lic_comm['categorie'].str.contains("20 à 24"), "20-24", lic_comm['tranche_age'])
    lic_comm['tranche_age'] = np.where(lic_comm['categorie'].str.contains("25 à 29"), "25-29", lic_comm['tranche_age'])
    lic_comm['tranche_age'] = np.where((lic_comm['categorie'].str.contains("30 à 34")) |
                                       (lic_comm['categorie'].str.contains("35 à 39")), "30-39", lic_comm['tranche_age'])

    lic_comm['tranche_age'] = np.where((lic_comm['categorie'].str.contains("40 à 44")) |
                                   (lic_comm['categorie'].str.contains("45 à 49")), "40-49", lic_comm['tranche_age'])

    lic_comm['tranche_age'] = np.where((lic_comm['categorie'].str.contains("50 à 54")) |
                                   (lic_comm['categorie'].str.contains("55 à 59")), "50-59", lic_comm['tranche_age'])

    lic_comm['tranche_age'] = np.where((lic_comm['categorie'].str.contains("60 à 64")) |
                                       (lic_comm['categorie'].str.contains("65 à 69")) |
                                       (lic_comm['categorie'].str.contains("70 à 74")) |
                                       (lic_comm['categorie'].str.contains("75 à 79")) |
                                       (lic_comm['categorie'].str.contains("80 à 99")), "60-99", lic_comm['tranche_age'])
    
    #Ajout de la variable sexe :
    lic_comm['sexe'] = np.where(lic_comm['categorie'].str.contains("F"), "F", lic_comm['sexe'])
    lic_comm['sexe'] = np.where(lic_comm['categorie'].str.contains("H"), "H", lic_comm['sexe'])

    # Ajouter les variables de population du département (base pop_dep) à la base lic_comm via la variable département :
    lic_comm = pd.merge(lic_comm, pop_dep, left_on='Département', right_on='Code département', how='left')

    return(lic_comm)